### Instituto Tecnológico de Costa Rica (ITCR)
### Escuela de Computación
### Curso: Inteligencia Artificial
 
### Tercer tarea programada 2022-I

Profesora: María Auxiliadora Mora


#Estudiantes
Roy Chavarría Garita - 2018034199 \
Jonder Hernández Gutiérrez - 2018203660




In [ ]:
#Bibliotecas
from matplotlib import pyplot
import numpy as np
import math
import matplotlib.pyplot as plt
from math import e
import torch
import plotly.graph_objects as go

from numpy import vstack
from numpy import sqrt
from numpy import array
from pandas import read_csv, notnull

# Pregunta 3
  # Gráficas estadísticas
import seaborn as sns
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_

#Graficar en 3d - pregunta #1
from sympy import symbols
from sympy.plotting import plot3d



# Linear regression
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression